<h4>Automated Soft Skills detection in IT Job Applications</h4>

**Preprocesar los datos de entrenamiento (el archivo CSV adjunto):**<br>
1. Limpiar y normalizar el texto de las postulaciones (quitar signos de puntuación, convertir a minúsculas, etc.)
2. Extraer las soft skills asociadas a cada postulación para tener las etiquetas de entrenamiento.

In [1]:
import pandas as pd
import string
import re

# Leer el archivo CSV
df = pd.read_csv('label_data.csv', sep=';')

# Normalizar texto 
df['text'] = df['text'].str.lower() # convertir a minúsculas
df['text'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-z0-9\s]', '', x)) # remover puntuación

# Extraer las labels de soft skills
df['soft_skills'] = df['soft_skills'].str.lower()
# df['soft_skills'] = df['soft_skills'].str.split(',')
df['soft_skills'] = df['soft_skills'].apply(lambda x: eval(x) if pd.notna(x) else [])


# Now check for cells with empty lists
empty_lists = df['soft_skills'].apply(lambda x: len(x) == 0)
df_filtered = df[~empty_lists]

# Imprimir algunos ejemplos
print(df['text'].head())
print(df['soft_skills'].head())

0    buscamos un data scientist altamente motivado ...
1    como data scientist en nuestro equipo particip...
2    estamos en busca de un data scientist apasiona...
3    nete a nuestro equipo como data scientist y co...
4    estamos contratando un data scientist con habi...
Name: text, dtype: object
0    [comunicación, trabajo en equipo, habilidades ...
1                                       [comunicación]
2             [resolución de problemas, adaptabilidad]
3                    [comunicación, trabajo en equipo]
4                             [habilidades analíticas]
Name: soft_skills, dtype: object


In [2]:
# Verificar si hay celdas vacías en la columna 'soft_skills'
celdas_vacias = df['soft_skills'].isnull().any()

# Imprimir el resultado
print(f'¿Hay celdas vacías en la columna "soft_skills"? {celdas_vacias}')

¿Hay celdas vacías en la columna "soft_skills"? False


In [3]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

# Asegúrate de descargar los recursos necesarios para nltk
nltk.download('punkt')

def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    return tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Importar librerías
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import string
import re

# Leer el archivo CSV
df = pd.read_csv('label_data.csv', sep=';')

# Eliminar postulaciones sin soft skills asociadas
df['soft_skills'] = df['soft_skills'].apply(lambda x: eval(x) if pd.notna(x) else [])
df = df[df['soft_skills'].apply(len) > 0]

# Normalizar texto 
df['text'] = df['text'].str.lower()  # convertir a minúsculas
df['text'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-z0-9\s]', '', x))  # remover puntuación

# Tokenizar el texto
vectorizer = TfidfVectorizer(tokenizer=tokenize_text) 

# Crear vectores de texto con TF-IDF
X = vectorizer.fit_transform(df['text'])

# Convertir etiquetas a formato binario
mlb = MultiLabelBinarizer()
y_bin = mlb.fit_transform(df['soft_skills'])

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size=0.2)

# Entrenar un modelo Naive Bayes con OneVsRest
nb = MultinomialNB()
model = OneVsRestClassifier(nb)

# Entrenar modelo
model.fit(X_train, y_train) 

# Evaluar performance
print(model.score(X_test, y_test))


0.0


c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\multiclass.py:84: UserWarning: Label not 5 is present in all training examples.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\multiclass.py:84: UserWarning: Label not 6 is present in all training examples.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\multiclass.py:84: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\multiclass.py:84: UserWarning: Label not 25 is present in all training examples.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\multicla

In [5]:
y_test

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 

In [6]:
from sklearn.model_selection import cross_val_score

# Utilizar validación cruzada
scores = cross_val_score(model, X_train, y_train, cv=5)  # Puedes ajustar el número de pliegues (folds) según sea necesario

# Imprimir las puntuaciones de validación cruzada
print("Puntuaciones de Validación Cruzada:", scores)
print("Precisión Promedio:", scores.mean())


Puntuaciones de Validación Cruzada: [0. 0. 0. 0. 0.]
Precisión Promedio: 0.0


c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\multiclass.py:84: UserWarning: Label not 5 is present in all training examples.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\multiclass.py:84: UserWarning: Label not 6 is present in all training examples.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\multiclass.py:84: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\multiclass.py:84: UserWarning: Label not 25 is present in all training examples.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\multiclass.py:84: UserWarning: Label not 27 is present in all training examples.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\multiclass.py:84: UserWarning: Label not 5 is pres

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

# # Asegúrate de tener la función tokenize_text definida
# def tokenize_text(text):
#     # Implementa la tokenización según sea necesario
#     return text.split()  # Ejemplo simple, tokeniza por espacios en blanco

# Convertir etiquetas de clase a formato binario
mlb = MultiLabelBinarizer()
y_train_encoded = mlb.fit_transform(y_train)
y_test_encoded = mlb.transform(y_test)

# Ajustar el vectorizador TF-IDF en los datos de entrenamiento
tfidf = TfidfVectorizer(tokenizer=tokenize_text)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Convertir los documentos a minúsculas después de la vectorización
X_train_lower = [doc.lower() for doc in X_train]
X_test_lower = [doc.lower() for doc in X_test]

# Importar modelos a probar
models = [
    ('Naive Bayes', MultinomialNB()),
    ('Regresión Logística', LogisticRegression()), 
    ('SVM', SVC()),
    ('Red Neuronal', MLPClassifier())
]

# Evaluar cada modelo
results = []
for name, model in models:
    # Ajustar el modelo al conjunto de entrenamiento
    model.fit(X_train_tfidf, y_train_encoded)
    # Predecir con el conjunto de prueba
    pred = model.predict(X_test_tfidf)
    # Evaluar métrica
    accuracy = accuracy_score(y_test_encoded, pred)
    # Guardar resultado 
    results.append((name, accuracy))

# Ordenar resultados  
results.sort(key=lambda x: x[1], reverse=True)

print('Performance de modelos:')
for name, accuracy in results:
    print(f'{name}: {accuracy:.2%}')


c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


AttributeError: lower not found

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer

# Asegúrate de tener la función tokenize_text definida
tfidf = TfidfVectorizer(tokenizer=tokenize_text)  

# Convertir etiquetas de clase a listas
y_train = y_train.apply(lambda x: eval(x) if pd.notna(x) else [])
y_test = y_test.apply(lambda x: eval(x) if pd.notna(x) else [])

# Codificar etiquetas con LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convertir etiquetas de habilidades blandas a vectores binarios
mlb = MultiLabelBinarizer()
y_train_encoded = mlb.fit_transform(y_train)
y_test_encoded = mlb.transform(y_test)

# Ajustar el vectorizador TF-IDF en los datos de entrenamiento
X_train_tfidf = tfidf.fit_transform(X_train)

# Importar modelos a probar
models = [
    ('Naive Bayes', MultinomialNB()),
    ('Regresión Logística', LogisticRegression()), 
    ('SVM', SVC()),
    ('Red Neuronal', MLPClassifier())
]

# Evaluar cada modelo
results = []
for name, model in models:
    if name in ['SVM', 'Regresión Logística']:
        # Ajustar el modelo al conjunto de entrenamiento
        model.fit(X_train_tfidf, y_train_encoded)
        # No es necesario ajustar el modelo aquí, ya que se ajustó previamente en el código
        pred = model.predict(X_test_tfidf)
    else:
        # Ajustar el modelo al conjunto de entrenamiento
        model.fit(X_train_tfidf, y_train_encoded)
        # Transformar las etiquetas de prueba sin ajustar
        pred = model.predict(X_test_tfidf)
    # Evaluar métrica
    accuracy = accuracy_score(y_test_encoded, pred)
    # Guardar resultado 
    results.append((name, accuracy))

# Ordenar resultados  
results.sort(key=lambda x: x[1], reverse=True)

print('Performance de modelos:')
for name, accuracy in results:
    print(f'{name}: {accuracy:.2%}')


AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder

# Ajustar el vectorizador TF-IDF en los datos de entrenamiento
tfidf = TfidfVectorizer(tokenizer=tokenize_text)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Convertir etiquetas de clase a listas
y_train = y_train.apply(lambda x: eval(x) if pd.notna(x) else [])
y_test = y_test.apply(lambda x: eval(x) if pd.notna(x) else [])

# Codificar etiquetas con MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train_encoded = mlb.fit_transform(y_train)
y_test_encoded = mlb.transform(y_test)

# Importar modelos a probar
models = [
    ('Naive Bayes', MultinomialNB()),
    ('Regresión Logística', LogisticRegression()), 
    ('SVM', SVC()),
    ('Red Neuronal', MLPClassifier())
]

# Evaluar cada modelo
results = []
for name, model in models:
    if name in ['SVM', 'Regresión Logística']:
        # Ajustar el modelo al conjunto de entrenamiento
        model.fit(X_train_tfidf, y_train_encoded)
        # No es necesario ajustar el modelo aquí, ya que se ajustó previamente en el código
        pred = model.predict(X_test_tfidf)
    else:
        # Ajustar el modelo al conjunto de entrenamiento
        model.fit(X_train_tfidf, y_train_encoded)
        # Transformar las etiquetas de prueba sin ajustar
        pred = model.predict(X_test_tfidf)
    # Evaluar métrica
    accuracy = accuracy_score(y_test_encoded, pred)
    # Guardar resultado 
    results.append((name, accuracy))

# Ordenar resultados  
results.sort(key=lambda x: x[1], reverse=True)

print('Performance de modelos:')
for name, accuracy in results:
    print(f'{name}: {accuracy:.2%}')


AttributeError: lower not found